In [ ]:
NLP Practical 3

In [33]:
import numpy as np
import os
from random import shuffle
import re
import sklearn as sk
import matplotlib.pyplot as plt
import random

In [34]:
import urllib.request
import zipfile
import lxml.etree

In [35]:
import tensorflow as tf

In [36]:
with zipfile.ZipFile('ted_en-20160408.zip', 'r') as z:
    doc = lxml.etree.parse(z.open('ted_en-20160408.xml', 'r'))
    
raw_text = doc.xpath('//content/text()')
raw_label = doc.xpath('//head/keywords/text()')

del doc

In [37]:
talk_sentences = []
talknum = len(raw_text)

for i in range(talknum):
    temp = re.sub(r'\([^)]*\)', '', raw_text[i])
    temp = re.sub(r'\n', '', raw_text[i])
    temp = temp.split('.')
    talk_sentences.append(temp)

In [38]:
print(talk_sentences[0][1])

To me the real, real solution to quality growth is figuring out the balance between two activities: exploration and exploitation


In [39]:
talk_sentence_word = []

for talk in talk_sentences:
    temp = []
    for sent in talk:
        tokens = re.sub(r"[^a-z0-9]+", " ", sent.lower()).split()
        temp.append(tokens)#
    talk_sentence_word.append(temp)

In [40]:
talk_sentence_word[0][1]

['to',
 'me',
 'the',
 'real',
 'real',
 'solution',
 'to',
 'quality',
 'growth',
 'is',
 'figuring',
 'out',
 'the',
 'balance',
 'between',
 'two',
 'activities',
 'exploration',
 'and',
 'exploitation']

In [41]:
freq = {}

for talk in talk_sentence_word:
    for sent in talk:
        for word in sent:
            if word in freq:
                freq[word] += 1
            else:
                freq[word] = 1

In [42]:
from collections import Counter

counts_ted_top1000 = []
words_top_ted = []

for word, count in Counter(freq).most_common(1000):
    counts_ted_top1000.append(count)
    words_top_ted.append(word)

In [43]:
talk_sentence_word_nostop = talk_sentence_word

for stop in words_top_ted[:200]:
    for talk in talk_sentence_word_nostop:
        for sent in talk:
            for word in sent:
                if word == stop:
                    sent.remove(word)

In [44]:
talk_sentence_word_nostop[0][1]

['real',
 'real',
 'solution',
 'quality',
 'growth',
 'figuring',
 'balance',
 'between',
 'activities',
 'exploration',
 'exploitation']

In [45]:
np.shape(talk_sentence_word_nostop)

(2085,)

In [46]:
input_label = []

for i in range(len(raw_label)):
    temp = re.sub(r' ', '', raw_label[i])
    input_label.append(temp.split(','))
    
#print(input_label_prepro)

In [47]:
labels_binary = []

for i in range(len(input_label)):
    temp = np.zeros(3)
    if 'technology' in '~'.join(input_label[i]):
        temp[0] = 1
    if 'entertainment' in '~'.join(input_label[i]):
        temp[1] = 1
    if 'design' in '~'.join(input_label[i]):
        temp[2] = 1
    labels_binary.append(temp)
        
print(input_label[9])
print(labels_binary[:10])

['talks', 'Senses', 'augmentedreality', 'brain', 'computers', 'creativity', 'cyborg', 'demo', 'design', 'engineering', 'entrepreneur', 'innovation', 'interfacedesign', 'invention', 'neuroscience', 'potential', 'prediction', 'productdesign', 'technology', 'visualizations']
[array([ 0.,  0.,  0.]), array([ 0.,  0.,  0.]), array([ 0.,  0.,  0.]), array([ 0.,  0.,  1.]), array([ 1.,  0.,  1.]), array([ 0.,  0.,  0.]), array([ 0.,  0.,  0.]), array([ 0.,  0.,  0.]), array([ 0.,  0.,  0.]), array([ 1.,  0.,  1.])]


In [48]:
temp = list(zip(talk_sentence_word_nostop, labels_binary))
random.shuffle(temp)
talk_sentence_word_nostop_shuffle, labels_binary_shuffle = zip(*temp)

In [49]:
labels_onehot = []
for i in range(len(input_label)):
    temp = np.zeros(8)
    temp[labels_binary[i][0]*4+labels_binary[i][1]*2+labels_binary[i][2]*1] = 1
    labels_onehot.append(temp)

/home/scratch/ms16lg2/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:4: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [50]:
labels_onehot_shuffle = []
for i in range(len(input_label)):
    temp = np.zeros(8)
    temp[labels_binary_shuffle[i][0]*4+labels_binary_shuffle[i][1]*2+labels_binary_shuffle[i][2]*1] = 1
    labels_onehot_shuffle.append(temp)

/home/scratch/ms16lg2/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:4: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [51]:
train_data = talk_sentence_word_nostop[:1835]
test_data = talk_sentence_word_nostop[1835:]

train_labels = labels_binary[:1835]
test_labels = labels_binary[1835:]

train_labels_onehot = labels_onehot[:1835]
test_labels_onehot = labels_onehot[1835:]

In [52]:
train_data_shuffle = talk_sentence_word_nostop_shuffle[:1835]
test_data_shuffle = talk_sentence_word_nostop_shuffle[1835:]

train_labels_shuffle = labels_binary_shuffle[:1835]
test_labels_shuffle = labels_binary_shuffle[1835:]

train_labels_onehot_shuffle = labels_onehot_shuffle[:1835]
test_labels_onehot_shuffle = labels_onehot_shuffle[1835:]

In [53]:
print(np.shape(talk_sentence_word_nostop))
print(np.shape(labels_binary))
print(np.shape(labels_onehot))
print(np.shape(train_data))
print(np.shape(train_labels))
print(np.shape(train_labels_onehot))
print(np.shape(test_data))
print(np.shape(test_labels))
print(np.shape(test_labels_onehot))

(2085,)
(2085, 3)
(2085, 8)
(1835,)
(1835, 3)
(1835, 8)
(250,)
(250, 3)
(250, 8)


In [54]:
print(np.shape(talk_sentence_word_nostop_shuffle))
print(np.shape(labels_binary_shuffle))
print(np.shape(labels_onehot_shuffle))
print(np.shape(train_data_shuffle))
print(np.shape(train_labels_shuffle))
print(np.shape(train_labels_onehot_shuffle))
print(np.shape(test_data_shuffle))
print(np.shape(test_labels_shuffle))
print(np.shape(test_labels_onehot_shuffle))

(2085,)
(2085, 3)
(2085, 8)
(1835,)
(1835, 3)
(1835, 8)
(250,)
(250, 3)
(250, 8)


In [55]:
check = np.asarray(train_labels_onehot)
print(check.sum(0) / check.sum())
check = np.asarray(test_labels_onehot)
print(check.sum(0) / check.sum())

[ 0.57493188  0.0746594   0.0719346   0.00980926  0.17711172  0.06376022
  0.01525886  0.01253406]
[ 0.292  0.108  0.164  0.024  0.224  0.112  0.032  0.044]


In [56]:
check = np.asarray(train_labels_onehot_shuffle)
print(check.sum(0) / check.sum())
check = np.asarray(test_labels_onehot_shuffle)
print(check.sum(0) / check.sum())

[ 0.53569482  0.07847411  0.08446866  0.01089918  0.18692098  0.07084469
  0.01961853  0.01307902]
[ 0.58   0.08   0.072  0.016  0.152  0.06   0.     0.04 ]


In [57]:
train_lenghts = []

for talk in train_data:
    N = 0
    for sent in talk:
        for word in sent:
            N += 1
    train_lenghts.append(N)

In [58]:
train_lenghts[:10]

[475, 807, 852, 493, 971, 823, 676, 692, 977, 622]

In [59]:
test_lenghts = []

for talk in test_data:
    N = 0
    for sent in talk:
        for word in sent:
            N += 1
    test_lenghts.append(N)

In [60]:
test_lenghts[:10]

[208, 1014, 826, 203, 1718, 241, 100, 979, 944, 1273]

In [61]:
train_talk_word = []

for talk in train_data:
    temp = []
    for sent in talk:
        for word in sent:
            temp.append(word)
    train_talk_word.append(temp)
    
print(len(train_talk_word[0]))

475


In [62]:
test_talk_word = []

for talk in test_data:
    temp = []
    for sent in talk:
        for word in sent:
            temp.append(word)
    test_talk_word.append(temp)
    
print(len(test_talk_word[0]))

208


In [63]:
test_talk_word[0][:10]

['fire',
 'nine',
 'days',
 'archive',
 '175',
 'films',
 '16',
 'millimeter',
 'negative',
 'books']

In [64]:
train_sent = []

for talk in train_data:
    for sent in talk:
        train_sent.append(sent)

In [65]:
from gensim.models import Word2Vec

In [66]:
model_ted = Word2Vec(train_sent, min_count=1, size=100)

In [67]:
train_talk_word_emb = []

for talk in train_talk_word:
    temp = []
    for word in talk:
        if word in model_ted.vocab.keys():
            temp.append(model_ted[word])
        else:
            temp.append(np.zeros(100))
    train_talk_word_emb.append(temp)

In [68]:
test_talk_word_emb = []

for talk in test_talk_word:
    temp = []
    for word in talk:
        if word in model_ted.vocab.keys():
            temp.append(model_ted[word])
        else:
            temp.append(np.zeros(100))
    test_talk_word_emb.append(temp)

In [69]:
train_lenghts[:10]

[475, 807, 852, 493, 971, 823, 676, 692, 977, 622]

In [74]:
np.shape(test_talk_word_emb)

(250,)

In [ ]:
train_talk_word_emb_exp = []

for i in range(len(train_talk_word_emb)):
    temp = []
    for j in range(max(train_lenghts)):
        if j <= (train_lenghts[i]-1):
            temp.append(train_talk_word_emb[i][j])
        else:
            temp.append(np.zeros(100))
    train_talk_word_emb_exp.append(temp)

In [ ]:
test_talk_word_emb_exp = []

for i in range(len(test_talk_word_emb)):
    temp = []
    for j in range(max(train_lenghts)):
        if j <= (test_lenghts[i]-1):
            temp.append(test_talk_word_emb[i][j])
        else:
            temp.append(np.zeros(100))
    test_talk_word_emb_exp.append(temp)

In [ ]:
np.shape(test_talk_word_emb_exp)

In [ ]:
np.shape(train_talk_word_emb_exp)

In [28]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.05)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

In [70]:
rnn_nodes = 100
hidden_nodes = 20

tf.reset_default_graph()

# NN inputs
x = tf.placeholder(tf.float32, shape=[None, 100, 1])
#xl = tf.placeholder(tf.float32, shape=[None])
y_ = tf.placeholder(tf.float32, shape=[None, 8])

# Defining RNN
cell = tf.nn.rnn_cell.LSTMCell(num_units=100, state_is_tuple=True)

h_rnn, last_states = tf.nn.dynamic_rnn(
    cell=cell,
    inputs=x,
    #sequence_length=xl,
    dtype=tf.float32)

h_bow = tf.reduce_mean(h_rnn, 0)

# First normal layer
W = weight_variable([100, hidden_nodes])
b = bias_variable([hidden_nodes])
h = tf.tanh(tf.matmul(h_bow, W) + b)

# Output Layer
V = weight_variable([hidden_nodes, 8])
c = bias_variable([8])
y = tf.nn.softmax(tf.matmul(h, V) + c)

In [71]:
# We'll use the cross entropy loss function 
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(y, y_))

# And classification accuracy
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# And the Adam optimiser
train_step = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(cross_entropy)

In [73]:
# Start a tf session and run the optimisation algorithm
sess = tf.Session()
sess.run(tf.initialize_all_variables())

input_data = train_talk_word_emb
input_labels = train_labels_onehot
epochs = 100
bs = 10
train_accuracy = np.zeros(epochs)
test_accuracy = np.zeros(epochs)
N = 0
C = 0

for epoch in range(epochs):
    for bat in range(len(input_data) // bs):
        print("hi")
        data_batch = input_data[bat * bs : (bat + 1) * bs]
        labels_batch = input_labels[bat * bs : (bat + 1) * bs]
        train_accuracy[epoch] += sess.run(accuracy, feed_dict={x: data_batch, y_: labels_batch}) / (len(input_data) // bs)
        test_accuracy[epoch] += sess.run(accuracy, feed_dict={x: test_bow, y_: test_labels_onehot}) / (len(input_data) // bs)
        sess.run(train_step, feed_dict={x: data_batch, y_: labels_batch})
    temp = list(zip(input_data, input_labels))
    random.shuffle(temp)
    input_data, input_labels = zip(*temp)

hi


ValueError: setting an array element with a sequence.

In [ ]:
train_lenghts = np.array(train_lenghts)
train_talk_word_emb_exp = np.array(train_talk_word_emb_exp)

tf.reset_default_graph()

cell = tf.nn.rnn_cell.LSTMCell(num_units=100, state_is_tuple=True)

outputs, last_states = tf.nn.dynamic_rnn(
    cell=cell,
    dtype=tf.float64,
    sequence_length=train_lenghts[:1],
    inputs=train_talk_word_emb_exp[:1])

result = tf.contrib.learn.run_n(
    {"outputs": outputs, "last_states": last_states},
    n=1,
    feed_dict=None)

#assert result[0]["outputs"].shape == (1, 100, 100)
print(result[0]["outputs"])

# Outputs for the second example past past length 6 should be 0
#assert (result[0]["outputs"][1,7,:] == np.zeros(cell.output_size)).all()

In [ ]:
tf.reset_default_graph()

# Create input data
X = np.random.randn(2, 10, 8)

# The second example is of length 6 
X[1,6:] = 0
X_lengths = [10, 6]

cell = tf.nn.rnn_cell.LSTMCell(num_units=64, state_is_tuple=True)

outputs, last_states = tf.nn.dynamic_rnn(
    cell=cell,
    dtype=tf.float64,
    sequence_length=X_lengths,
    inputs=X)

result = tf.contrib.learn.run_n(
    {"outputs": outputs, "last_states": last_states},
    n=1,
    feed_dict=None)

assert result[0]["outputs"].shape == (2, 10, 64)
print(result[0]["outputs"])

# Outputs for the second example past past length 6 should be 0
assert (result[0]["outputs"][1,7,:] == np.zeros(cell.output_size)).all()

In [ ]:
X = np.random.randn(2, 10, 8)

In [ ]:
X